In [1]:
#source: https://github.com/wiseodd/generative-models/blob/master/GAN/vanilla_gan/gan_pytorch.py

import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
#from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from torchvision import datasets, transforms

In [2]:
#load mnist data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

def create_mnist_dataset(data, labels, batch_size):
  def gen():
    for image, label in zip(data, labels):
        yield image, label
  ds = tf.data.Dataset.from_generator(gen, (tf.float32, tf.int32), ((28,28 ), ()))

  return ds.repeat().batch(batch_size)

#train and validation dataset with different batch size
mnist = create_mnist_dataset(x_train, y_train, 64)
#valid_dataset = create_mnist_dataset(x_test, y_test, 20)

11501568/11490434 [==============================] - 1s 0us/step


In [26]:
#mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

mb_size = 64
Z_dim = 100
#X_dim = mnist.train.images.shape[1]
#y_dim = mnist.train.labels.shape[1]
X_dim = 28
y_dim = 28
h_dim = 128
c = 0
lr = 1e-3

train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data/', train=True, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
                             batch_size=mb_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data/', train=False, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
                             batch_size=mb_size, shuffle=True)


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

In [27]:
#Generator
Wzh = xavier_init(size=[Z_dim, h_dim])
bzh = Variable(torch.zeros(h_dim), requires_grad=True)

Whx = xavier_init(size=[h_dim, X_dim])
bhx = Variable(torch.zeros(X_dim), requires_grad=True)


def G(z):
    h = torch.relu(z @ Wzh + bzh.repeat(z.size(0), 1))
    print(h.size())
    X = torch.sigmoid(h @ Whx + bhx.repeat(h.size(0), 1))
    print(X.size())
    return X

In [29]:
#Discriminator
Wxh = xavier_init(size=[X_dim, h_dim])
bxh = Variable(torch.zeros(h_dim), requires_grad=True)

Why = xavier_init(size=[h_dim, 1])
bhy = Variable(torch.zeros(1), requires_grad=True)


def D(X):
    lokjo = bxh.repeat(X.size(0), 1)
    print(lokjo.size())
    h = torch.relu(X @ Wxh + lokjo)
    print(h.size())
    y = torch.sigmoid(h @ Why + bhy.repeat(h.size(0), 1))
    print(y.size())
    return y


G_params = [Wzh, bzh, Whx, bhx]
D_params = [Wxh, bxh, Why, bhy]
params = G_params + D_params

In [6]:
def reset_grad():
    for p in params:
        if p.grad is not None:
            data = p.grad.data
            p.grad = Variable(data.new().resize_as_(data).zero_())


G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

ones_label = Variable(torch.ones(mb_size, 1))
zeros_label = Variable(torch.zeros(mb_size, 1))


In [30]:
#Training
#for it in range(20000):
it = 0
my_range = 20000
for epoch in range(int(my_range / 60000 + 1)):
    for _, data in enumerate(train_loader, 0):
        if it < my_range:
            # Sample data
            z = Variable(torch.randn(mb_size, Z_dim))
            #X, _ = mnist.train.next_batch(mb_size)
            #X = Variable(torch.from_numpy(X))
            X, _ = data

            print(X.size())

            # Dicriminator forward-loss-backward-update
            G_sample = G(z)
            D_real = D(X)
            D_fake = D(G_sample)

            D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
            D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
            D_loss = D_loss_real + D_loss_fake

            D_loss.backward()
            D_solver.step()

            # Housekeeping - reset gradient
            reset_grad()

            # Generator forward-loss-backward-update
            z = Variable(torch.randn(mb_size, Z_dim))
            G_sample = G(z)
            D_fake = D(G_sample)

            G_loss = nn.binary_cross_entropy(D_fake, ones_label)

            G_loss.backward()
            G_solver.step()

            # Housekeeping - reset gradient
            reset_grad()

            # Print and plot every now and then
            if it % 1000 == 0:
                print('Iter-{}; D_loss: {}; G_loss: {}'.format(it, D_loss.data.numpy(), G_loss.data.numpy()))

                samples = G(z).data.numpy()[:16]

                fig = plt.figure(figsize=(4, 4))
                gs = gridspec.GridSpec(4, 4)
                gs.update(wspace=0.05, hspace=0.05)

                #for i, sample in enumerate(samples):
                #    ax = plt.subplot(gs[i])
                #    plt.axis('off')
                #    ax.set_xticklabels([])
                #    ax.set_yticklabels([])
                #    ax.set_aspect('equal')
                #    plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

                if not os.path.exists('out/'):
                    os.makedirs('out/')

                #plt.savefig('out/{}.png'.format(str(c).zfill(3)), bbox_inches='tight')
                c += 1
                #plt.close(fig)

            it = it + 1

torch.Size([64, 1, 28, 28])
torch.Size([64, 128])
torch.Size([64, 28])
torch.Size([64, 128])


RuntimeError: The size of tensor a (28) must match the size of tensor b (64) at non-singleton dimension 2